In [1]:
from engine import Tensor
import nn
import functional as F
from optim import SGD
import numpy as np

In [2]:
# use torchvision to get mnist data
import torch
import torchvision 
from torchvision import transforms

MLP definition

In [3]:
class MLP(nn.Module):
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = []
        for i in range(len(nouts)):
            self.layers.append(nn.Linear(sz[i], sz[i+1]))
            if i != len(nouts) - 1:
                self.layers.append(nn.Sigmoid())
    
    def __repr__(self):
        return f"MLP of [{', '.join(str(layer) for layer in self.layers)}]"

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

Training

In [4]:
train_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=True,
                                           transform=transforms.Compose([
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                           download=True)

In [5]:
# put training data in lists for preprocessing
xs = []
ys = []

for x, y in train_dataset:
    xs.append(x)
    ys.append(y)

len(xs), len(ys)

(60000, 60000)

In [6]:
# convert Pytorch tensors to my Tensors
xs = list(map(torch.flatten, xs))
xs = list(map(lambda x: x.numpy(), xs))
xs = list(map(Tensor, xs))

In [7]:
model = MLP(784, [16, 16, 10])
optimizer = SGD(model.parameters(), lr=0.1)

# training loop
max_epochs = 3
for epoch in range(max_epochs):
    for it, (x, y) in enumerate(zip(xs, ys)):
        # forward pass
        ypred = model(x)
        loss = F.cross_entropy(ypred, y)
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        
        # update
        optimizer.step()
        
        if it % 10000 == 0:
            print(f"epoch: {epoch}, it: {it}, loss: {loss}")

epoch: 0, it: 0, loss: Tensor(data=[3.557336496510241])
epoch: 0, it: 10000, loss: Tensor(data=[0.19869085823822127])
epoch: 0, it: 20000, loss: Tensor(data=[0.03536561253556344])
epoch: 0, it: 30000, loss: Tensor(data=[0.006098864294135648])
epoch: 0, it: 40000, loss: Tensor(data=[0.010459694777903948])
epoch: 0, it: 50000, loss: Tensor(data=[0.14227055232271368])
epoch: 1, it: 0, loss: Tensor(data=[0.4744455272267569])
epoch: 1, it: 10000, loss: Tensor(data=[0.032191507779018344])
epoch: 1, it: 20000, loss: Tensor(data=[0.17142187821564928])
epoch: 1, it: 30000, loss: Tensor(data=[0.012866718766937613])
epoch: 1, it: 40000, loss: Tensor(data=[0.017674090837408448])
epoch: 1, it: 50000, loss: Tensor(data=[0.022112146472283788])
epoch: 2, it: 0, loss: Tensor(data=[0.4031863070890516])
epoch: 2, it: 10000, loss: Tensor(data=[0.031748325883355646])
epoch: 2, it: 20000, loss: Tensor(data=[0.011675474284432906])
epoch: 2, it: 30000, loss: Tensor(data=[0.022570750121275067])
epoch: 2, it: 4

Testing

In [8]:
test_dataset = torchvision.datasets.MNIST(root='./data',
                                          train=False,
                                          transform=transforms.Compose([
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                          download=True)

In [9]:
# put test data in lists for preprocessing
test_xs = []
test_ys = []

for x, y in test_dataset:
    test_xs.append(x)
    test_ys.append(y)

len(test_xs), len(test_ys)


(10000, 10000)

In [10]:
# convert PyTorch tensors to my Tensor
test_xs = list(map(torch.flatten, test_xs))
test_xs = list(map(lambda x: x.numpy(), test_xs))
test_xs = list(map(Tensor, test_xs))

In [11]:
# evaluate accuracy on test set
correct = 0
total = 0
for x, y in zip(test_xs, test_ys):
    logits = model(x)
    pred = np.argmax(logits.data)
    correct += (y == pred)
    total += 1

print(f"accuracy: {correct / total}")

accuracy: 0.8861
